In [1]:
from pulp import *
import pulp
import numpy as np


In [11]:
def solve_lp(T, vehicle_start, vehicle_end, charger_loc, charger_co2s, starting_battery, ending_battery, speed, num_chargers_per_station, max_rate_of_charge, battery_per_mile, grid_size):
    D = 2
    size = grid_size

    # create a linear program
    prob = LpProblem("CVRP with Charging Stations", LpMinimize)

    # convert x into a lp variable
    x = LpVariable.dicts("x",[(i, j, k) for i in range(len(vehicle_start)) for j in range(T) for k in range(D)], lowBound=0, upBound=size, cat=LpInteger)

    # variable for tracking the current charge level
    b = LpVariable.dicts("b",[(i, j) for i in range(len(vehicle_start)) for j in range(T)], lowBound=0, upBound=100, cat=LpInteger)

    # variable for holding the distance traveled
    d = LpVariable.dicts("d",[(i, j, k) for i in range(len(vehicle_start)) for j in range(T-1) for k in range(D)], lowBound=0, upBound=None, cat=LpInteger)

    # variable for holding the how much charge is added
    c = LpVariable.dicts("c",[(i, ci, t) for ci in range(len(charger_loc)) for i in range(len(vehicle_start)) for t in range(T-1)], lowBound=0, upBound=max_rate_of_charge, cat=LpInteger)
    
    # binary variable for tracking whether c is positive
    c_cond = LpVariable.dicts("c_cond",[(i, ci, t) for ci in range(len(charger_loc)) for i in range(len(vehicle_start)) for t in range(T-1)], lowBound=0, upBound=1, cat=LpBinary)

    # absolute variable for difference between cars and charging stations
    c_diff = LpVariable.dicts("c_diff",[(i, ci, t, j) for j in range(2) for ci in range(len(charger_loc)) for i in range(len(vehicle_start)) for t in range(T-1)], lowBound=0, upBound=None, cat=LpInteger)
    """
    Model charging for the battery. We want to enforce the constraint such that when a car is close enough to one of the charging locations, then it can get charged there. 
    Each car should only be able to connect to one charger and one charger should only be able to serve up to num_chargers_per_station cars
    """

    # cars start with a specific battery level
    for i in range(len(vehicle_start)):
        prob += b[i, 0] == starting_battery[i]

    # cars end with at least a specific battery level
    for i in range(len(vehicle_start)):
        prob += b[i, T-1] >= ending_battery[i]

    # battery must be above 10 at all times
    for i in range(len(vehicle_start)):
        for t in range(T):
            prob += b[i, t] >= 10

    for i in range(len(vehicle_start)):
        for ci in range(len(charger_loc)):
            for t in range(T-1):
                c_diff[i, ci, t, 0] >= (x[i, t, 0] - charger_loc[ci][0])
                c_diff[i, ci, t, 0] >= -(x[i, t, 0] - charger_loc[ci][0])

                c_diff[i, ci, t, 1] >= (x[i, t, 1] - charger_loc[ci][1])
                c_diff[i, ci, t, 1] >= -(x[i, t, 1] - charger_loc[ci][1])

                # c[i, ci, t] <= (max_rate_of_charge - (c_diff[i, ci, t, 0] + c_diff[i, ci, t, 1])) * c_cond[i, ci, t] + 0 * (1 - c_cond[i, ci, t])

                # c_cond[i, ci, t] = (c_diff[i, ci, t, 0] + c_diff[i, ci, t, 1]) == 0
                # print(c_cond[i, ci, t].varValue())
                # c[i, ci, t] <= (c_cond[i, ci, t] - 1) * max_rate_of_charge
                # if the car is close enough to the charger, then it can be charged
                # prob += c_cond[i, ci, t] >= (x[i, t, 0] - charger_loc[ci][0]) * (x[i, t, 0] - charger_loc[ci][0]) + (x[i, t, 1] - charger_loc[ci][1]) * (x[i, t, 1] - charger_loc[ci][1])

    obj1 = 0
    time_objective = 0
    # one charger can only serve num_chargers_per_station cars
    for ci in range(len(charger_loc)):
        for t in range(T-1):
            charger_consumption = lpSum(c[i, ci, t] for i in range(len(vehicle_start)))
            obj1 += charger_consumption * charger_co2s[ci]
            prob += charger_consumption <= num_chargers_per_station * max_rate_of_charge

    # only one car can be connected to a charger
    for i in range(len(vehicle_start)):
        for t in range(T-1):
            prob += lpSum(c[i, ci, t] for ci in range(len(charger_loc))) <= max_rate_of_charge

    # enforce a speed constraint for lp variables
    for i in range(len(vehicle_start)):
        for t in range(T-1):
            prob += x[i, t+1, 1] - x[i, t, 1] <= d[i, t, 1]
            prob += x[i, t, 1] - x[i, t+1, 1] <= d[i, t, 1]

            prob += x[i, t+1, 0] - x[i, t, 0] <= d[i, t, 0]
            prob += x[i, t, 0] - x[i, t+1, 0] <= d[i, t, 0]

            prob += d[i, t, 0] + d[i, t, 1] <= speed
            time_objective += (d[i, t, 0] + d[i, t, 1]) * t
            prob += b[i, t+1] == b[i, t] + lpSum(c[i, ci, t] for ci in range(len(charger_loc))) - (d[i, t, 0] + d[i, t, 1]) * battery_per_mile

    # enforce the final location constraint
    for i in range(len(vehicle_start)):
        prob += x[i, T-1, 0] == vehicle_end[i, 0]
        prob += x[i, T-1, 1] == vehicle_end[i, 1]

    # enforce the start location constraint
    for i in range(len(vehicle_start)):
        prob += x[i, 0, 0] == vehicle_start[i, 0]
        prob += x[i, 0, 1] == vehicle_start[i, 1]

    obj2 = lpSum(lpSum(lpSum(d[i, t-1, k] for k in range(2)) for t in range(1, T)) for i in range(len(vehicle_start)))

    prob += obj1 + obj2 + time_objective

    prob.solve()
    x_out = np.zeros((len(vehicle_start), T, D))
    for i in range(len(vehicle_start)):
        for t in range(T):
            for k in range(D):
                x_out[i,t, k] = x[i,t, k].varValue

    # create c out
    c_out = np.zeros((len(vehicle_start), len(charger_loc), T-1))
    for i in range(len(vehicle_start)):
        for ci in range(len(charger_loc)):
            for t in range(T-1):
                c_out[i, ci, t] = c[i, ci, t].varValue


    # create b out
    b_out = np.zeros((len(vehicle_start), T))
    for i in range(len(vehicle_start)):
        for t in range(T):
            b_out[i, t] = b[i, t].varValue

    # create c cond out
    c_cond_out = np.zeros((len(vehicle_start), len(charger_loc), T-1))
    for i in range(len(vehicle_start)):
        for ci in range(len(charger_loc)):
            for t in range(T-1):
                c_cond_out[i, ci, t] = c_cond[i, ci, t].value()

    objective = pulp.value(prob.objective)
    print("Objective = {}".format(objective))

    return prob, objective, x_out, c_out, b_out, c_cond_out, vehicle_start, vehicle_end
    

In [12]:
size = 20
vehicle_start = np.array([[0,0]])
vehicle_end = np.array([[size, size]])
charger_loc = np.array([[10,10], [5, 5], [90, 90]])
charger_co2s = np.array([0.1, 0.2, 0.3])
start_battery = [30]
ending_battery = [10]
T = 30
prob, objective, x_out, c_out, b_out, c_cond_out, starts, ends =  solve_lp(T, vehicle_start, vehicle_end, charger_loc, charger_co2s, start_battery, ending_battery, speed=10, num_chargers_per_station=5, max_rate_of_charge=3, battery_per_mile=1, grid_size=size)
prob.status

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/akshgarg/opt/anaconda3/envs/ML/lib/python3.7/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ld/prbck6fj4tn7tt5rq173lfj40000gp/T/b9f3a809bb0a45e5abfda4d0f7eda854-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/ld/prbck6fj4tn7tt5rq173lfj40000gp/T/b9f3a809bb0a45e5abfda4d0f7eda854-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 331 COLUMNS
At line 1766 RHS
At line 2093 BOUNDS
At line 2329 ENDATA
Problem MODEL has 326 rows, 235 columns and 819 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 118 - 0.00 seconds
Cgl0004I processed model has 167 rows, 168 columns (168 integer (0 of which binary)) and 530 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0012I Integer solution of 118 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 se

1

In [22]:
import pandas as pd

display(df)


,Unnamed: 0,Start Address,End Address,Start Latitude,Start Longitude,End Latitude,End Longitude
0,0,SF,SFO Airport,37.779026,-122.419906,37.622452,-122.383989
1,1,SF,Stanford,37.779026,-122.419906,37.426541,-122.170305
2,2,SF,San Diego,37.779026,-122.419906,32.717420,-117.162773


37.622452


,Unnamed: 0,Start Address,End Address,Start Latitude,Start Longitude,End Latitude,End Longitude
0,0,SF,SFO Airport,0,0,1.000000,0.000000
1,1,SF,Stanford,0,0,0.960059,0.040926
2,2,SF,San Diego,0,0,0.000000,1.000000


In [48]:
df = pd.read_csv('df.csv')

df['Start Latitude'] = df['Start Latitude'].astype(float)
df['Start Longitude'] = df['Start Longitude'].astype(float)
df['End Latitude'] = df['End Latitude'].astype(float)
df['End Longitude'] = df['End Longitude'].astype(float)

display(df)

print(df['End Latitude'].max())
def normalize_column(df, col):
    if df[col].max() == df[col].min():
        df[col] = 0
        return df
    
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df

df = pd.read_csv('df.csv')
df = normalize_column(df, 'Start Latitude')
df = normalize_column(df, 'Start Longitude')
df = normalize_column(df, 'End Latitude')
df = normalize_column(df, 'End Longitude')

starts = df[['Start Latitude', 'Start Longitude']].values
ends = df[['End Latitude', 'End Longitude']].values

size = 20

starts = (starts * size).astype(int)
ends = (ends * size).astype(int)

vehicle_start = starts
vehicle_end = ends
num_chargers = 10
charger_loc = np.random.randint(0, size, (num_chargers, 2))
charger_co2s = np.random.rand(num_chargers)

start_battery = [25] * len(vehicle_start)
ending_battery = [10] * len(vehicle_start)
T = 30

prob, objective, x_out, c_out, b_out, c_cond_out, starts, ends =  solve_lp(T, vehicle_start, vehicle_end, charger_loc, charger_co2s, start_battery, ending_battery, speed=2, num_chargers_per_station=5, max_rate_of_charge=3, battery_per_mile=1, grid_size=size)
print(prob.status)

# compute the total distance traveled
total_distance = 0
for i in range(len(vehicle_start)):
    for t in range(T-1):
        total_distance += np.linalg.norm(x_out[i, t, :] - x_out[i, t+1, :])


# compute the total co2
co2_obj = 0
for ci in range(len(charger_loc)):
    for t in range(T-1):
        for i in range(len(vehicle_start)):
            co2_obj += c_out[i, ci, t] * charger_co2s[ci]
            

time_to_solution = 0
# Find the first timestamp after which the x_out doesn't change
for t in range(T-1):
    if np.linalg.norm(x_out[:, t, :] - x_out[:, t+1, :]) > 0:
        time_to_solution = t
    else:
        break


print("Total distance traveled = {}".format(total_distance))
print("Total co2 = {}".format(co2_obj))
print("Time to solution = {}".format(time_to_solution))

,Unnamed: 0,Start Address,End Address,Start Latitude,Start Longitude,End Latitude,End Longitude
0,0,SF,SFO Airport,37.779026,-122.419906,37.622452,-122.383989
1,1,SF,Stanford,37.779026,-122.419906,37.426541,-122.170305
2,2,SF,San Diego,37.779026,-122.419906,32.717420,-117.162773


37.622452


/Users/akshgarg/opt/anaconda3/envs/ML/lib/python3.7/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/akshgarg/opt/anaconda3/envs/ML/lib/python3.7/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ld/prbck6fj4tn7tt5rq173lfj40000gp/T/3f723cc2fe4f4e3dbe15ff362f779613-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/ld/prbck6fj4tn7tt5rq173lfj40000gp/T/3f723cc2fe4f4e3dbe15ff362f779613-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1012 COLUMNS
At line 8969 RHS
At line 9977 BOUNDS
At line 11292 ENDATA
Problem MODEL has 1007 rows, 1314 columns and 4284 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 322.866 - 0.00 seconds
Cgl0004I processed model has 493 rows, 486 columns (486 integer (0 of which binary)) and 1583 elements
Cbc0012I Integer solution of 322.86624 found by DiveCoefficient after 0 iterations and 0 nodes (0.03 seconds)
Cbc0001I Search completed

In [47]:
c_out

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [41]:
# compute the total distance traveled
total_distance = 0
for i in range(len(vehicle_start)):
    for t in range(T-1):
        total_distance += np.linalg.norm(x_out[i, t, :] - x_out[i, t+1, :])


# compute the total co2
co2_obj = 0
for ci in range(len(charger_loc)):
    for t in range(T-1):
        for i in range(len(vehicle_start)):
            co2_obj += c_out[i, ci, t] * charger_co2s[ci]
            

time_to_solution = 0
# Find the first timestamp after which the x_out doesn't change
for t in range(T-1):
    if np.linalg.norm(x_out[:, t, :] - x_out[:, t+1, :]) > 0:
        time_to_solution = t
    else:
        break

In [42]:
x_out

array([[[ 0.,  0.],
        [10.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.],
        [20.,  0.]],

       [[ 0.,  0.],
        [10.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.],
        [19.,  0.]

array([[10,  0],
       [ 9,  0],
       [ 0, 10]])